In [2]:
# install libraries
import influxdb_client
import pandas as pd
import secret

In [3]:
# code to read data from influxdb

# variables
bucket = "random"
org = "Empati Limited"
token = secret.api_key
# Store the URL of your InfluxDB instance
url="https://eu-central-1-1.aws.cloud2.influxdata.com/"

# instantiate the client
client = influxdb_client.InfluxDBClient(
   url=url,
   token=token,
   org=org
)

# instantiate the query client
query_api = client.query_api()

# query written in InfluxQL
query = 'from(bucket:"random")\
|> range(start: -10m)\
|> filter(fn:(r) => r._measurement == "Power")\
|> filter(fn:(r) => r._field == "value")'

# submit the query
result = query_api.query(org=org, query=query)

results = []
for table in result:
    for record in table.records:
        results.append((record.get_field(), record.get_value()))

print(results)

[('value', 3.325357), ('value', 3.252319), ('value', 3.333591), ('value', 3.265402), ('value', 3.319153), ('value', 3.2784), ('value', 3.307606), ('value', 3.290919), ('value', 3.301646), ('value', 3.308112), ('value', 3.282965), ('value', 3.324476), ('value', 3.276978), ('value', 3.325152), ('value', 3.262301), ('value', 3.31837), ('value', 3.255982), ('value', 3.317088), ('value', 3.248441), ('value', 3.321384), ('value', 3.244683), ('value', 3.322323), ('value', 3.236502), ('value', 3.333812), ('value', 3.234127), ('value', 3.328216), ('value', 3.234948), ('value', 3.32733), ('value', 3.244745), ('value', 3.311259), ('value', 3.266894), ('value', 3.281998), ('value', 3.304215), ('value', 3.259813), ('value', 3.328218), ('value', 3.241921), ('value', 4.011397), ('value', 4.134586), ('value', 4.232809), ('value', 4.167645), ('value', 4.189505), ('value', 4.1982), ('value', 4.156235), ('value', 4.225678), ('value', 4.13647), ('value', 4.243755), ('value', 4.123164), ('value', 4.24836),

In [28]:
# separate the result into time, appliance and power

old_results = []
new_results = []
for table in result:
    for record in table.records:
        if record.values['appliance'] == 'OLD':
            old_results.append((record.values['_time'], record.values['_value']))
        else:
            new_results.append((record.values['_time'], record.values['_value']))

# put the results into a dataframe
old_df = pd.DataFrame(old_results, columns=['time', 'power'])
new_df = pd.DataFrame(new_results, columns=['time', 'power'])



In [30]:
old_df['datetime'] = pd.to_datetime(old_df['time'])
old_df['power'] = pd.to_numeric(old_df['power'])
old_df = old_df.set_index('datetime')
old_df = old_df.drop(columns=['time'])

new_df['datetime'] = pd.to_datetime(new_df['time'])
new_df['power'] = pd.to_numeric(new_df['power'])
new_df = new_df.set_index('datetime')
new_df = new_df.drop(columns=['time'])



In [31]:
old_df

,power
datetime,
2023-12-15 16:11:56+00:00,0.081779
2023-12-15 16:11:57+00:00,0.082310
2023-12-15 16:11:58+00:00,0.081101
2023-12-15 16:11:59+00:00,0.083104
2023-12-15 16:12:00+00:00,0.080626
...,...
2023-12-15 16:21:49+00:00,0.078911
2023-12-15 16:21:50+00:00,0.077853
2023-12-15 16:21:51+00:00,0.079068


In [ ]:
# create function to call in the generation (OLD) data
lookback = '10m'
def get_generation_data(str: lookback):
    
